In [1]:
import datetime
import random
import math
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal


In [2]:
def make_5min_bars(tick_data):
    #Resample to get 5min bars
    five_min_data = pd.DataFrame(
        tick_data['last'].resample('5Min', loffset=datetime.timedelta(minutes=5)).ohlc())
        
    #Create RTH Calendar
        
    #We hack the NYSE Calendar extending the close until 4:15
    class CMERTHCalendar(mcal.exchange_calendar_nyse.NYSEExchangeCalendar):
        @property
        def close_time(self):
            return datetime.time(16, 15)
        
    nyse = CMERTHCalendar()
    schedule = nyse.schedule(start_date=five_min_data.index.min(), 
                             end_date=five_min_data.index.max())
        
    #Filter out those bars that occur during RTH
    five_min_data['dates'] = pd.to_datetime(five_min_data.index.to_datetime().date)
    five_min_data['valid_date'] = five_min_data['dates'].isin(schedule.index)
    five_min_data['valid_time'] = False
    during_rth = five_min_data['valid_date'] & \
        (five_min_data.index > schedule.loc[five_min_data['dates'],'market_open']) & \
        (five_min_data.index <= schedule.loc[five_min_data['dates'],'market_close'])
    five_min_data.loc[during_rth, 'valid_time'] = True
    five_min_data = five_min_data[five_min_data['valid_time'] == True]
    five_min_data.drop(['dates','valid_date','valid_time'], axis=1, inplace=True)
        
    #Add ema
    five_min_data['ema'] = five_min_data['close'].ewm(span=20, min_periods=20).mean()

    #Reset index
    five_min_data.reset_index(inplace=True)
        
    #Add column for number of seconds elapsed in trading day
    five_min_data['sec'] = (five_min_data['date'].values 
                            - five_min_data['date'].values.astype('datetime64[D]')) / np.timedelta64(1,'s')

    #Calculate sin & cos time
    #24hr time is a cyclical continuous feature
    seconds_in_day = 24*60*60
    five_min_data['sin_time'] = np.sin(2*np.pi*five_min_data['sec']/seconds_in_day)
    five_min_data['cos_time'] = np.cos(2*np.pi*five_min_data['sec']/seconds_in_day)

    five_min_data.drop('sec', axis=1, inplace=True)
        
    return five_min_data

In [3]:
td = pd.read_feather('../data/processed/ES_tick.feather')
td = td[td['date'] > '2017-07-29']
#Create Index from date column
td.index = td['date']
td.drop(labels=['date'],axis=1,inplace=True)

In [56]:
fd = make_5min_bars(td)

#Add columns tracking reward for buy/sell actions and number of bars until reward
fd.loc[:,'sell_r'] = 0
fd.loc[:,'sell_b'] = 0
fd.loc[:,'buy_r'] = 0
fd.loc[:,'buy_b'] = 0

In [11]:
fd.head()

,date,open,high,low,close,ema,sin_time,cos_time,sell_r,sell_b,buy_r,buy_b
0,2017-07-31 09:35:00-04:00,2474.75,2475.75,2474.00,2475.50,NaN,-0.402747,-0.915311,0,0,0,0
1,2017-07-31 09:40:00-04:00,2475.25,2476.00,2473.75,2475.50,NaN,-0.422618,-0.906308,0,0,0,0
2,2017-07-31 09:45:00-04:00,2475.75,2475.75,2474.50,2474.75,NaN,-0.442289,-0.896873,0,0,0,0
3,2017-07-31 09:50:00-04:00,2474.50,2475.00,2473.50,2473.75,NaN,-0.461749,-0.887011,0,0,0,0
4,2017-07-31 09:55:00-04:00,2474.00,2474.25,2472.75,2472.75,NaN,-0.480989,-0.876727,0,0,0,0


In [57]:
def simulate(start_index, action, order_price):
    position = 'flat'
    working_order = None
    reward = 0
    for i, r in fd[start_index+1:].iterrows():
         # Get tick data for the current bar
        ts_end = r['date']
        ts_start = ts_end - pd.Timedelta(minutes=5)
        ticks = td[(td.index > ts_start) & \
                   (td.index <= ts_end)]
        
        if position == 'flat':
            if action == 'buy':
                working_order = 'buy'
                target_price = order_price + 1
                stop_price = order_price - 1
            elif action == 'sell':
                working_order = 'sell'
                target_price = order_price - 1
                stop_price = order_price + 1
                
        
        for t_i, t_r in ticks.iterrows():
            price = t_r['last']
            if position == 'flat':
                if working_order == 'buy':
                    if price < order_price:
                        position = 'long'
                        working_order = None
                        order_price = 0
                elif working_order == 'sell':
                    if price > order_price:
                        position = 'short'
                        working_order = None
                        order_price = 0
            elif position == 'long':
                if price > target_price:
                    position = 'flat'
                    reward = 50
                elif price <= stop_price:
                    position = 'flat'
                    reward = -50
            elif position == 'short':
                if price < target_price:
                    position = 'flat'
                    reward = 50
                elif price >= stop_price:
                    position = 'flat'
                    reward = -50
        
        if position == 'flat':
            break
            
    return reward, i - (start_index+1)
    

In [58]:
for index, row in fd[:-1].iterrows():
    fd.loc[index, 'buy_r'], fd.loc[index, 'buy_b'] = simulate(index, 'buy', row['low'])
    fd.loc[index, 'sell_r'], fd.loc[index, 'sell_b'] = simulate(index, 'sell', row['high'])

In [59]:
fd.tail(100)

,date,open,high,low,close,ema,sin_time,cos_time,sell_r,sell_b,buy_r,buy_b
9905,2018-01-24 14:45:00-05:00,2841.25,2842.50,2841.00,2842.25,2838.815696,-0.896873,0.442289,-50,0,0,0
9906,2018-01-24 14:50:00-05:00,2842.50,2845.25,2842.50,2844.00,2839.309439,-0.887011,0.461749,0,0,-50,2
9907,2018-01-24 14:55:00-05:00,2844.25,2844.75,2841.75,2842.25,2839.589492,-0.876727,0.480989,0,0,0,0
9908,2018-01-24 15:00:00-05:00,2842.25,2843.00,2841.75,2842.50,2839.866684,-0.866025,0.500000,0,0,-50,0
9909,2018-01-24 15:05:00-05:00,2842.50,2842.75,2839.50,2839.75,2839.855571,-0.854912,0.518773,0,0,0,0
9910,2018-01-24 15:10:00-05:00,2839.75,2841.50,2839.75,2841.25,2839.988374,-0.843391,0.537300,50,0,50,0
9911,2018-01-24 15:15:00-05:00,2841.25,2842.00,2839.00,2840.00,2839.989481,-0.831470,0.555570,0,0,50,0
9912,2018-01-24 15:20:00-05:00,2839.75,2841.25,2838.75,2840.00,2839.990483,-0.819152,0.573576,0,0,0,0
9913,2018-01-24 15:25:00-05:00,2839.75,2841.00,2839.25,2840.75,2840.062818,-0.806445,0.591310,50,0,0,0
9914,2018-01-24 15:30:00-05:00,2840.75,2841.50,2839.25,2840.50,2840.104454,-0.793353,0.608761,0,0,-50,0


In [60]:
fd.to_feather('../data/processed/ES_5mintrading.feather')